In [ ]:
!pip install pandas
!pip install psycopg2


import pandas as pd
import psycopg2
import numpy as np


sales_df = pd.read_csv(r'AbbasStores\raw_data\sales_data.csv')



In [ ]:
sales_df.drop_duplicates()
sales_df.drop_duplicates(subset = 'Transaction_ID', inplace=True)
sales_df.reset_index

In [ ]:
fill_columns= ['Feedback', 'Gender', 'Product_Category', 'Shipping_Method' ,'Phone']
for col in fill_columns:
    sales_df[col]=sales_df[col].fillna('Not Provided')


In [226]:
sales_df['Total_Purchases' ] = sales_df['Total_Purchases' ].fillna(0)

In [228]:
sales_df['Transaction_ID'] = sales_df['Transaction_ID'].astype(str).str.replace('.0', "", regex=False)
sales_df['Customer_ID'] = sales_df['Customer_ID'].astype(str).str.replace('.0', "", regex = False) 
sales_df['Phone']  = sales_df['Phone'].astype(str).str.replace('.0', "", regex =False) 
sales_df ['Year'] =  sales_df ['Year'] .astype(str).str.replace('.0', "", regex =False) 
sales_df['Zipcode'] = sales_df['Zipcode'].astype(str).str.replace('.0', "", regex =False) 
sales_df['Age'] = sales_df['Age'].fillna('0').astype('int64')
sales_df['Date'] =pd.to_datetime(sales_df['Date'])
sales_df['Ratings'] =pd.to_numeric(sales_df['Ratings'], errors ='coerce').fillna(0)
sales_df['Time'] = pd.to_datetime(sales_df['Time'], format="%H:%M:%S", errors='coerce').dt.time


In [229]:
new_IDs = ['Product_ID', 'Date_ID', 'Address_ID', 'Feedback_ID']
for col in new_IDs:
    sales_df[col] = sales_df.index + 1

In [ ]:
customers_df = sales_df[['Customer_ID', 'Name', 'Email','Phone','Age','Gender', 'Income','Customer_Segment']].drop_duplicates(subset = 'Customer_ID').reset_index (drop= True)


product_df = sales_df[['Product_ID', 'products', 'Product_Category', 'Product_Brand', 'Product_Type']].drop_duplicates().reset_index(drop=True)
product_df.rename (columns = {'products': 'Product_Name'}, inplace =True)


address_df = sales_df[['Address_ID','Address', 'City','State','Zipcode', 'Country']].drop_duplicates().reset_index(drop=True)



date_df = sales_df[['Date_ID','Date','Year','Month','Time']].drop_duplicates().reset_index(drop=True)



feedback_df = sales_df[['Feedback_ID', 'Product_ID','Customer_ID', 'Transaction_ID','Ratings']].drop_duplicates().reset_index(drop=True)




transactions_df = sales_df[['Transaction_ID','Customer_ID','Product_ID','Address_ID','Date_ID',
                            'Total_Purchases' , 'Amount','Total_Amount', 'Payment_Method',
                           'Order_Status', 'Shipping_Method']].drop_duplicates().reset_index(drop=True)

In [265]:
customers = customers_df.to_csv(r'AbbasStores\processed_data\customers.csv', index =False)
products = product_df.to_csv(r'AbbasStores\processed_data\products.csv', index =False)
address = address_df.to_csv(r'AbbasStores\processed_data\address.csv', index =False)
date = date_df.to_csv(r'AbbasStores\processed_data\date.csv', index =False)
feedback = feedback_df.to_csv(r'AbbasStores\processed_data\feedback.csv', index =False)
transactions = transactions_df.to_csv(r'AbbasStores\processed_data\transactions.csv', index =False)

In [ ]:

top_5 = sales_df.sort_values(by = 'Total_Amount', ascending=False)
print (top_5[['Customer_ID','Name' ,'Total_Amount']])

In [ ]:
avg_rating = sales_df.pivot_table(values = 'Ratings',
                                  index ='Product_Type',
                                  aggfunc = 'mean')

print(avg_rating)

In [ ]:
rating_array = sales_df['Ratings'].to_numpy()
print(rating_array)
print (rating_array.mean())

In [270]:
def get_db_connection():
    connection =psycopg2.connect (host = 'localhost',
                 database = 'AbbasStores',
                 user = 'postgres',
                 password = 'postgres')
    return connection

conn = get_db_connection()
cursor = conn.cursor()

def create_tables ():
    conn = get_db_connection()
    cursor = conn.cursor()
    table_query = ''' CREATE SCHEMA IF NOT EXISTS crafts;

                        DROP TABLE IF EXISTS crafts.customers CASCADE;
                        DROP TABLE IF EXISTS crafts.products CASCADE;
                        DROP TABLE IF EXISTS crafts.address CASCADE;
                        DROP TABLE IF EXISTS crafts.date CASCADE;
                        DROP TABLE IF EXISTS crafts.feedback CASCADE;
                        DROP TABLE IF EXISTS crafts.transactions CASCADE;


                        CREATE TABLE IF NOT EXISTS crafts.customers
                        (Customer_ID INT NOT NULL PRIMARY KEY,
                        Name TEXT,
                        Email TEXT,
                        Phone TEXT,
                        Age INT,
                        Gender TEXT,
                        Income TEXT,
                        Customer_Segment TEXT
                
                        );


                        
                        CREATE TABLE IF NOT EXISTS crafts.products
                        (Product_ID INT NOT NULL PRIMARY KEY,
                        Product_Name TEXT,
                        Product_Category TEXT,
                        Product_Brand TEXT,
                        Product_Type TEXT
                        
                        );

                      
                    
                        CREATE TABLE IF NOT EXISTS crafts.date
                        (Date_ID INT NOT NULL PRIMARY KEY,
                        Date DATE,
                        Year TEXT,
                        Month TEXT,
                        Time TIME
                        
                        );


                        CREATE TABLE IF NOT EXISTS crafts.address
                        (Address_ID INT NOT NULL PRIMARY KEY,
                        Address TEXT,
                        City TEXT,
                        State TEXT,
                        Zipcode TEXT,
                        Country TEXT
                        
                        );


                        CREATE TABLE IF NOT EXISTS crafts.transactions
                        (Transaction_ID INT NOT NULL PRIMARY KEY,
                        Customer_ID  INT,
                        Product_ID INT,
                        Address_ID INT,
                        Date_ID INT,
                        Total_Purchases FLOAT,
                        Amount FLOAT,
                        Total_Amount FLOAT,
                        Payment_Method TEXT,
                        Order_Status TEXT,
                        Shipping_Method TEXT,
                       
                        FOREIGN KEY (Customer_ID) REFERENCES crafts.customers (Customer_ID),
                        FOREIGN KEY (Product_ID) REFERENCES crafts.products (Product_ID),
                        FOREIGN KEY (Address_ID) REFERENCES crafts.address (Address_ID),
                        FOREIGN KEY (Date_ID) REFERENCES crafts.date (Date_ID)
                        
                        );



                        
                        CREATE TABLE IF NOT EXISTS crafts.feedback
                        (Feedback_ID INT NOT NULL PRIMARY KEY,
                        Product_ID INT,
                        Customer_ID INT,
                        Transaction_ID INT,
                        Ratings FLOAT,
                        FOREIGN KEY (Product_ID) REFERENCES crafts.products  (Product_ID),
                        FOREIGN KEY (Customer_ID) REFERENCES crafts.customers (Customer_ID),
                        FOREIGN KEY (Transaction_ID) REFERENCES crafts.transactions (Transaction_ID)
                        ); 
                        '''
    
    cursor.execute(table_query)

    conn.commit()
    cursor.close()
    conn.close()



In [271]:
create_tables()

In [277]:

import csv
def load_table_data(csv_path):
    conn =get_db_connection()
    cursor = conn.cursor()
    with open (csv_path, 'r') as file:
        reader =csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute('''  INSERT INTO crafts.feedback(Feedback_ID, Product_ID, Customer_ID, Transaction_ID, Ratings)
                             VALUES (%s, %s, %s, %s, %s)''', row)
                        
                       
    conn.commit()
    cursor.close()

csv_file_path = r'AbbasStores\processed_data\feedback.csv'
load_table_data(csv_file_path)